# Лабораторная работа 7. Обучение без учителя

Результат лабораторной работы − отчет. Мы предпочитаем принимать отчеты в формате ноутбуков IPython (ipynb-файл). Постарайтесь сделать ваш отчет интересным рассказом, последовательно отвечающим на вопросы из заданий. Помимо ответов на вопросы, в отчете также должен быть код, однако чем меньше кода, тем лучше всем: нам − меньше проверять, вам — проще найти ошибку или дополнить эксперимент. При проверке оценивается четкость ответов на вопросы, аккуратность отчета и кода.

## Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 15 баллов. Сдавать задание после указанного срока сдачи нельзя. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов и понижают карму. Если вы нашли решение какого-то из заданий в открытом источнике, необходимо прислать ссылку на этот источник (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, нам необходима ссылка на источник).

## EM-алгоритм

### Краудсорсинг

Разметка данных — одна из самых трудозатратных задач в машинном обучении. Краудсорсинг позволяет распределить эту задачу на тысячи исполнителей, каждый из которых подготавливает небольшую часть датасета (подробнее <a href="https://academy.yandex.ru/posts/chto-takoe-kraudsorsing-i-pochemu-emu-nuzhno-uchitsya">тут</a>).

Пользователи могут допускать ошибки при разметке, кроме того, среди пользователей могут быть боты. Если мы попросим разметить каждый объект только одного пользователя, то с большой вероятностью получим не достаточно качественную разметку. Обычно каждый объект размечают несколько пользователей.

Результаты разметки нужно обработать. Самый простой метод *голос большинства*. Он заключается в том, что для каждого объекта нужно взять тот класс, который чаще всего ставили пользователи данному объекту. Это достаточно хороший метод, но он не учитывает различные особенности пользователей. Далее рассмотрим метод, который позволяет оценивать вероятность того, что разметчик ошибся.

### Метод Дэвида-Скина (Dawid, Skene, 1979)

Мы имеем в качестве данных $n_{ik}^u$ &mdash; количество раз, при которых разметчик $u \in U$ поставил класс $k \in K$ объекту $i \in I$ (возможно, разметчик видел этот объект несколько раз). Обозначим $Y_{ik} = I\{\text{объект $i$ класса $k$}\}$, это наши латентные величины. 

В качестве параметров имеем
* $\pi_{k\ell}^u$ &mdash; вероятность того, что разметчик $u$ поставил класс $\ell$ вместо правильного класса $k$. 
* $\rho_k$ &mdash; вероятность класса $k$.

Поймём, какой будет функция неполного правдоподобия в этой задаче. Прежде всего,

$$p_{\pi,p}(N, Y) = \prod_{i\in I}p(N_i, Y_i),$$

Если $k$ - номер класса $i$-го объекта, то

$$p(N_i, Y_i)=\underbrace{p(\text{объект $i$ класса $k$})}_{=\rho_k}p(N_i\mid\text{объект $i$ класса $k$})$$

(значения $Y_{it}$ однозначно определяются номером истинного класса, поэтому справа $Y_i$ пропадает). Далее, мы считаем, что разметчики действуют независимо, поэтому

$$p(N_i\mid\text{объект $i$ класса $k$}) = \prod_{u\in U}p(N_i^u\mid\text{объект $i$ класса $k$}).$$

Разберёмся с величиной $p(N_i^u\mid\text{объект $i$ класса $k$})$. Она отвечает за то, какие классы $u$-й разметчик ставил $i$-му объекту. Мы считаем, что встречи разметчика с объектом упорядочены по времени, тогда

$$p(\text{$u$-й разметчик отнёс $i$-й объект к классам $k'_1,\ldots,k'_r$}\mid\text{объект $i$ класса $k$}) =$$

$$=\prod_{s}p(\text{в $s$-ю встречу с $i$-м объектом $u$-й разметчик отнёс его к классу $k'_s$}\mid\text{объект $i$ класса $k$})$$

Эту вероятность можно переписать в виде

$$\prod_{\ell \in K} \left( \pi_{k\ell}^u \right)^{n_{i\ell}^u},$$ - ненормированная вероятность

а итоговое неполное правдоподобие предстаёт в виде

$$p_{\pi,p}(N, Y) = \prod_{i\in I}\prod_{k \in K} \left( \rho_k \prod_{u\in U} \prod_{\ell \in K} \left( \pi_{k\ell}^u \right)^{n_{i\ell}^u} \right)^{Y_{ik}}$$

Его нам нужно максимизировать по $\pi$ и $\rho$

**Пояснение к формуле:** 

Вне больших скобок фиксируются объект и его класс, сама скобка возводится в степень 1, если рассматривается правильный класс объекта, и в степень 0 иначе. Внутри сначала записана вероятность того, что объект имеет данный класс, а затем &mdash; перебор по всем пользователям и всем классам, которые мог поставить данный пользователь. Наконец, записывается вероятность того, что пользователь нашему объекту поставил некоторый класс, которая возводится в степень того, сколько раз он поставил этот класс. Например, если пользователь видел изображение котика 5 раз, при этом 3 раза он сказал, что котик, а два раза &mdash; песик, то вероятность $\pi_{cat,cat}^u$ для данного котика учтется 3 раза, а вероятность $\pi_{cat,dog}^u$ &mdash; 2 раза.

**Задание 1 (2 балл)**

Распишите итерационную процедуру EM-алгоритма и значение ELBO в методе Дэвида-Скина.

In [1]:
import pandas as pd
import numpy as np
from scipy.special import logsumexp, softmax

In [2]:
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix, dok_matrix
from scipy.special import softmax

class DawidSkene:
    def __init__(self, crowd_labels: pd.DataFrame):
        self.crowd_labels = crowd_labels

        # Create a mapping from user/item/label to their corresponding indices
        self.user_mapping = {user: i for i, user in enumerate(crowd_labels['C1'].unique())}
        self.item_mapping = {item: i for i, item in enumerate(crowd_labels['C2'].unique())}
        self.label_mapping = {label: i for i, label in enumerate(crowd_labels['C3'].unique())}

        self.grouped = crowd_labels.groupby(['C1', 'C2', 'C3']).size().reset_index(name='counts')

        # Initialize the joint_occurrence dictionary
        self.joint_occurrence = {} #with respect to item for faster E-step
        self.joint_occurrence_u = {} #with respect to user for faster M-step
        for _, row in self.grouped.iterrows():
            i = self.user_mapping[row['C1']]
            j = self.item_mapping[row['C2']]
            k = self.label_mapping[row['C3']]
            if j not in self.joint_occurrence:
                self.joint_occurrence[j] = {}
            self.joint_occurrence[j][(i, k)] = row['counts']
            if i not in self.joint_occurrence_u:
                self.joint_occurrence_u[i] = {}
            self.joint_occurrence_u[i][(j, k)] = row['counts']


        # Initialize item-label probabilities uniformly
        #self.rho = np.full(len(self.label_mapping), np.log(1 / len(self.label_mapping)))
        self.rho = np.random.rand(len(self.label_mapping))
        self.rho = np.log(self.rho/np.sum(self.rho))

        # Initialize labeler confusion matrices uniformly. On fix user
        self.pi = np.random.rand(len(self.user_mapping), len(self.label_mapping), len(self.label_mapping))
        self.pi = np.log(self.pi/np.sum(self.pi, axis=2, keepdims=True))
        # self.pi = np.full(
        #     (len(self.user_mapping), len(self.label_mapping), len(self.label_mapping)),
        #     np.log(1 / len(self.label_mapping) ** 2)
        # )
        # Initialize the item_label_prob array
        self.q = np.zeros((len(self.item_mapping), len(self.label_mapping)))

    def e_step(self):
        # Compute the item_label_prob array using the current parameter estimates
        for j in range(len(self.item_mapping)):
            sum_pi = np.zeros((len(self.label_mapping),))
            for k in range(len(self.label_mapping)):
                for (i, el), v in self.joint_occurrence[j].items():
                    sum_pi[k] += self.joint_occurrence[j][(i, el)] * self.pi[i, k, el]
            self.q[j, :] = softmax(self.rho + sum_pi, axis=0)

    def m_step(self):
        self.rho = np.log(np.mean(self.q, axis=0))
        for u in range(len(self.user_mapping)):
            sum_q = np.full((len(self.label_mapping), len(self.label_mapping)), 1e-8) # np.zeros((len(self.label_mapping), len(self.label_mapping)))
            denom = 0
            for (i,r), v in self.joint_occurrence_u[u].items():
                denom += v
                sum_q[:, r] += v * self.q[i, :]
            self.pi[u] = np.log(sum_q / denom) # np.log(np.clip(sum_q / denom, 1e-6, None))

    def predict(self):
        data = []
        for item, i in self.item_mapping.items():
            data.append({'object': item, 'prediction': np.argmax(self.q[i, :])})
        df = pd.DataFrame(data, columns=['object', 'prediction'])
        return df
        # df = pd.DataFrame(columns=['item', 'label'])
        # for item, i in self.item_mapping.items():
        #     df = df.append({'item': item, 'label': np.argmax(self.q[i, :])}, ignore_index=True)
        # return df

    # Format the DataFrame to match the required output format
    def run(self, n_iter):
        for _ in range(n_iter):
            self.e_step()
            self.m_step()
        return self.predict()


**Задание 2 (3 балла)** 

Реализуйте следующие методы агрегации результатов разметки в краудсорсинге:

* голосование по большинству;
* метод Дэвида-Скина.

Оба метода должны возвращать вероятность принадлежности объекта каждому из классов (итоговая метка получается выбором класса с наибольшей оценкой вероятности). 

Заметим, что метод голосования по большинству можно реализовать с помощью одной агрегирующей функцией `pandas.crosstab`, а метод Дэвида-Скина основывается на EM-алгоритме (при реализации стоит учесть, что EM-алгоритм сходится в локальные оптимумы, то есть его стоит запускать из разных начальных приближений).

Примените эти два метода к датасетам *Toloka Aggregation Relevance 2* и *Toloka Aggregation Relevance 5*, которые можно скачать <a href="https://toloka.ai/ru/datasets">тут</a>, и сравните их между собой. 

Обратите внимание, что в последнем датасете 5 различных меток, причем некоторые объекты в датасете имеют не один, а несколько правильных ответов. Любой из таких ответов считается правильным.

**В Я.Контест нужно загрузить только .csv файлы в следующем формате.**

In [3]:
def majority_vote_agg(df: pd.DataFrame) -> pd.DataFrame:
    """
    Aggregates crowd-sourced labels using majority voting.
    
    Args:
    df (pd.DataFrame): DataFrame containing columns ['C2', 'C1', 'C3'].
    
    Returns:
    pd.DataFrame: DataFrame with columns ['object', 'prediction'].
    """
    # Create a crosstab of votes
    crosstab = pd.crosstab(df[item_col], df[label_col])

    # Determine the class with the highest probability
    crosstab['prediction'] = crosstab.idxmax(axis=1).astype(int)

    # Rename columns to match the required format
    crosstab.reset_index(inplace=True)
    crosstab.rename(columns={item_col: 'object'}, inplace=True)

    # Select only the necessary columns
    result = crosstab[['object', 'prediction']]

    return result


# Load the datasets
crowd_labels = pd.read_csv('download/TlkAgg2/crowd_labels.tsv', sep='\t', header=None,
                           names=['C1', 'C2', 'C3'])
gold_labels = pd.read_csv('download/TlkAgg2/golden_labels.tsv', sep='\t', header=None, names=['C2', 'C3'])

# Define column names based on the actual data
item_col = 'C2'  # task_id
labeler_col = 'C1'  # worker_id
label_col = 'C3'  # label

# Apply Majority Voting
majority_result = majority_vote_agg(crowd_labels)
majority_result.sort_index(inplace=True)
majority_result.to_csv('majority_result.csv', sep=',', index=False)
print("\nMajority Voting Result:")
print(majority_result.head())



Majority Voting Result:
C3 object  prediction
0      t0           1
1      t1           1
2     t10           1
3    t100           0
4   t1000           0


In [4]:
majority_result

C3,object,prediction
0,t0,1
1,t1,1
2,t10,1
3,t100,0
4,t1000,0
...,...,...
99314,t9995,1
99315,t9996,0
99316,t9997,0
99317,t9998,0


In [5]:
crowd_labels = pd.read_csv('download/TlkAgg2/crowd_labels.tsv', sep='\t', header=None, names=['C1', 'C2', 'C3'])
crowd_labels

,C1,C2,C3
0,w851,t30685,1
1,w6991,t30008,0
2,w2596,t36316,0
3,w5507,t15145,1
4,w2982,t44785,1
...,...,...,...
475531,w4660,t62250,1
475532,w6630,t46626,0
475533,w4605,t93513,1
475534,w1928,t29002,0


In [6]:
# Apply Dawid-Skene Method
dawid_skene = DawidSkene(crowd_labels)
result_df = dawid_skene.run(n_iter=5)

# Format the DataFrame to match the required output format
formatted_result = result_df.rename(columns={item_col: 'object', label_col: 'prediction'})[['object', 'prediction']]
formatted_result['prediction'] = formatted_result['prediction'].astype(int)
formatted_result = formatted_result.sort_values(by='object')

# Display the first few rows to verify
print("\nDawid-Skene Result for TlkAgg2 (First 5 Rows):")
print(formatted_result.head())

# Optionally, save the result to a CSV file
formatted_result.to_csv('dawid_skene_results.csv', index=False, sep=',')


Dawid-Skene Result for TlkAgg2 (First 5 Rows):
   object  prediction
0  t30685           1
1  t30008           1
2  t36316           1
3  t15145           1
4  t44785           1


In [11]:
formatted_result.sort_values(by='object').head(10)

,object,prediction
52887,t0,1
29893,t1,1
13040,t10,1
63391,t100,1
22532,t1000,1
71287,t10000,1
37868,t10001,1
57988,t10002,1
14406,t10003,1
39225,t10004,1


In [7]:

def majority_vote_agg5(df: pd.DataFrame) -> pd.DataFrame:
    """
    Aggregates crowd-sourced labels using majority voting.
    
    Args:
    df (pd.DataFrame): DataFrame containing columns ['worker_id', 'task_id', 'label'].
    
    Returns:
    pd.DataFrame: DataFrame with columns ['object', 'prediction'].
    """
    # Define column names explicitly
    item_col = 'task_id'
    label_col = 'label'

    # Create a crosstab of votes
    crosstab = pd.crosstab(df[item_col], df[label_col])

    # Determine the class with the highest probability
    crosstab['prediction'] = crosstab.idxmax(axis=1).astype(int)

    # Rename columns to match the required format
    crosstab.reset_index(inplace=True)
    crosstab.rename(columns={item_col: 'object'}, inplace=True)

    # Select only the necessary columns
    result = crosstab[['object', 'prediction']]

    return result


# Load the datasets for TlkAgg5
crowd_labels = pd.read_csv('download/TlkAgg5/tlk_agg5_crowd_labels.tsv', sep='\t', header=None,
                           names=['worker_id', 'task_id', 'label'])
gold_labels = pd.read_csv('download/TlkAgg5/tlk_agg5_golden_labels.tsv', sep='\t', header=None,
                          names=['task_id', 'label'])

# Apply Majority Voting
majority_result_tlkagg5 = majority_vote_agg5(crowd_labels)
majority_result_tlkagg5.sort_index(inplace=True)
majority_result_tlkagg5.head()

FileNotFoundError: [Errno 2] No such file or directory: 'download/TlkAgg5/tlk_agg5_crowd_labels.tsv'

In [ ]:
majority_result_tlkagg5.head()

In [ ]:
majority_result_tlkagg5.tail()


**Задание 3 (1 балл)** 

Попробуйте в методе Дэвида-Скина в качестве начального приближения вероятностей классов для каждого объекта подавать те вероятности, которые посчитаны методом голосования по большинству, и провести сначала M-шаг.

In [ ]:
class DawidSkeneWithMVInit(DawidSkene):
    def initialize_params(self):

# your code here

Далее мы будем работать с таблицей `vacancies.csv` и нам нужно подготовить данные для работы, а именно сделайте следующее:
    
- Разбейте данные на обучающую выборку (строки, не содержащие метки кластеров) и тестовую (строки, содержащие метки кластеров)
- Предобработайте текст, содержащийся в колонках *name* и *description* (уберите артефакты, нормализуйте и т.д.)

### Тематическое моделирование

Тематическое моделирование заключается в поиске тем $T$, которые хорошо бы описывали документы $D$ со словарём $W$. Большинство тематических моделей оперирует данными в формате "мешка слов", т.е. учитывают только частоты слов в документах, а не их порядок. Одной из простейших тематических моделей является [PLSA](https://en.wikipedia.org/wiki/Probabilistic_latent_semantic_analysis), которая приводит к задаче стохастического матричного разложения: 

$$F \approx \Phi \times \Theta$$
где
- $F_{W \times D}$— матрица распределений слов в документах (нормированные частоты)
- $\Phi_{W \times T}$ — матрица распределений слов в темах (модель)
- $\Theta_{T \times D}$ — матрица распределений тем в документах (результат применения модели к обучающим данным)

Можно сказать, что алгоритмы тематического моделирования производят мягкую бикластеризацию данных:
 - *мягкую*, так как объекты относятся не строго к одному кластеру, а к нескольким с разными вероятностями
 - *бикластеризацию*, так как модель одновременно кластеризует слова по темам и темы по документам.
 
 С вероятностной точки зрения, задача обучения модели PLSA ставится как максимизация неполного правдоподобия по параметам $\Phi$ и $\Theta$. ЕМ-алгоритм для модели PLSA заключается в повторении двух шагов:

- **Е-шаг** — оценка распределений тем для каждого слова в каждом документе по параметрам $\Phi$ и $\Theta$ (шаг 6);
- **М-шаг** — обновление параметров $\Phi$ и $\Theta$ на основе полученных оценок (шаги 7 и 9).

Существуют различные модификации итерационного процесса, позволяющие снизить расходы по памяти. В данном случае, мы избежим хранения трехмерной матрицы $p_{tdw}$, сразу пересчитывая $\Theta$ для текущего документа и аккумулируя счетчики $n_{wt}$ для последующего пересчета $\Phi$.

Псевдокод алгоритма записывается следующим образом:

1. Инициализировать $\phi_{wt}^0$ для всех $w \in W$, $t \in T$ и $\theta_{td}^0$ для всех $t \in T$, $d \in D$
2. Внешний цикл по итерациям $i = 1 ... max\_iter$:
3. $\quad$ $n_{wt}^i := 0$, $n_t^i := 0$ для всех $w \in W$ и $t \in T$ 
4. $\quad$ Внутренний цикл по документам $d \in D$  
5. $\qquad$ $Z_w := \sum_{t \in T} \phi_{wt}^{i-1}\theta_{td}^{i-1}$ для всех $w \in d$ $\cfrac{}{}$
6. $\qquad$ $p_{tdw} := \cfrac{ \phi_{wt}^{i-1}\theta_{td}^{i-1} }{ Z_w }$ (**E-шаг**)
7. $\qquad$ $\theta_{td}^{i} := \cfrac{ \sum_{w \in d} n_{dw} p_{tdw} }{ n_d }$ для всех $t \in T$ (**M-шаг**)
8. $\qquad$ Увеличить $n_{wt}^i$ и $n_t^i$ на $n_{dw} p_{tdw}$ для всех $w \in W$ и $t \in T$
9. $\quad \phi_{wt}^i := \cfrac{n_{wt}^i}{n_t^i}$ для всех $w \in W$ и $t \in T$ (**M-шаг**)

Обозначения:
 - $p_{tdw}$ — вероятность темы $t$ для слова $w$ в документе $d$
 - $\phi_{wt}$ — элемент матрицы $\Phi$, соответствующий вероятности слова $w$ в теме $t$
 - $\theta_{td}$ — элемент матрицы $\Theta$, соответствующий вероятности темы $t$ в документе $d$
 - $n_{wt}$ — элемент матрицы счётчиков отнесения слова $w$ к теме $t$ (путем нормирования этой матрицы получается матрица $\Phi$)
 - $Z_w$ — элемент вектора вспомогательных переменных, соответствующий слову $w$
 - $n_t$ — вектор нормировочных констант для матрицы $n_{wt}$
 - $n_d$ — вектор нормировочных констант для матрицы $n_{dw}$
 - $n$ — суммарное число слов в коллекции

Для оценивания качества построенной модели и контроля сходимости процесса обучения обычно используют [перплексию](http://www.machinelearning.ru/wiki/images/8/88/Voron-iip9-talk.pdf):

$$\mathcal{P} = \exp\bigg(- \frac{\mathcal{L}}{n} \bigg) = \exp\bigg(- \cfrac{1}{n}\sum_{d \in D}\sum_{w \in d} n_{dw} \ln \big(\sum_{t \in T}\phi_{wt}\theta_{td} \big)\bigg)$$

Это традиционная мера качества в тематическом моделировании, которая основана на правдоподобии модели $\mathcal{L}$. Число итераций $max\_iter$ в алгоритме обучения следует выбирать достаточным для того, чтобы перплексия перестала существенно убывать. Однако известно, что перплексия плохо отражает интерпретируемость найденных тем, поэтому помимо нее обычно используются дополнительные меры или экспертные оценки.

**Рекомендации к реализации:**

- При делении на нулевые значения нужно просто заменить частное на ноль.
- ЕМ-алгоритм стоит реализовывать с использованием векторных операций. Для проверки корректности реализации сперва можно написать скалярную версию, после чего векторизовать её, удостоверившись, что обе реализации дают одинаковый результат. Невекторизованный алгоритм может работать в сотни раз медленнее векторизованного, и его использование может привести к невозможности выполнения задания.
- Итерационный процесс следует начинать, инициализировав матрицы $\Phi$ и $\Theta$. Инициализация может быть случайной, важно не забыть отнормировать столбцы матриц.
- Неэффективная реализация перплексии может в разы замедлить работу алгоритма.

**Задание 4 (3 балла)** 

Реализуйте описанный выше ЕМ-алгоритм для модели *PLSA* и добавьте в вашу реализацию подсчёт перплексии. 

Примените ваш алгоритм к подготовленным ранее данным (объедините текст из колонок *name* и *description*), рассмотрев число тем T = 5, а также:

* Постройте график значения перплексии в зависимости от итерации (убедитесь в корректности реализации: график перплексии должен быть невозрастающим). 
* Выведите для каждой темы топ-20 наиболее вероятных слов.

Посмотрите внимательно на полученные темы. Как вам кажется, получились ли они интерпретируемыми?

In [ ]:
class PLSA:
    def __init__(self, counts: np.matrix, T: int):
        self.counts = counts
        self.T = T

        # your code here

        initialize_params()

    def initialize_params(self):
        self.Phi =  # your code here
        self.Theta =  # your code here

    def step(self):

    # your code here

    def perplexity(self) -> float:

# your code here

**Задание 5 (1 балл)** 

Рассмотрите большее число тем (10, 20) и несколько различных начальных приближений. Проанализируйте результаты и ответьте на следующие вопросы: 

- Mожно ли сказать, что интерпретируемость каждой темы изменяется с ростом их числа?
- Устойчив ли алгоритм к начальному приближению на примере идентичности топовых слов в соответствующих темах?
- Отражает ли перплексия качество получаемых моделей? В чём заключается причина хорошего/плохого соответствия?

## Кластеризация

**Задание 6 (2 балла)** 

В данном задание следуют сравнить между собой алгоритмы [KMeans](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html), [DBSCAN](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html) и [Birch](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.Birch.html) используя подготовленные ранее данные.

Поэксперементируйте с различными способами векторизации текста, например можно:

- использовать только *name* / *description*, объединить их в один текст или сконкатенировать векторные представления,
- использовать представление в виде мешка слов или какую-либо его модификацию,
- сократить размерность векторного представления, используя PCA или векторные представления слов,

или сделать что-то более интересное, что вы придумаете!

Выберете лучшую комбинацию векторизации и алгоритма кластеризации и визуализируйте полученные кластеры (например, воспользовавшись облаком тегов, или предложите свой способ). Обоснуйте почему вы считаете, что выбранный вами подход для решения задачи кластеризации вакансий является лучшим.

**Задание "со звездочкой" (1 балл)** 

Попробуйте обогатить векторное представление полученное из текстов другими признаками из таблицы `vacancies.csv` и добиться лучшей интерпретируемости кластеров.

## Частичное обучение

Часто у нас есть размеченная выборка только для небольшой части выборки. Тогда мы можем применить подходы _частичного обучения (semi-supervised learning)_. Более подробно про реализацию таких методов в sklearn можно прочитать в разделе [semi-supervised](http://scikit-learn.org/stable/modules/label_propagation.html#semi-supervised).

**Задание 7 (2 балла)** 

В этом задание нужно сделать следующее: 

- Разделите объекты, у которых существуют метки, на обучающую и тестовую выборки (при этом не обязательно делить в соотношении 70% на 30%). Обогатите обучающую выборку объектами без меток.
- Воспользовавшись опытом выполнения задания 6, возьмите "лучшее" векторное представление вакансий и обучите [LabelSpreading](http://scikit-learn.org/stable/modules/generated/sklearn.semi_supervised.LabelSpreading.html) (подберите лучшие параметры, опираясь на [F1 score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)).
- Попробуйте запустить алгоритм несколько раз, отмечая известными различные объекты, а также меняя пропорции разбиения, посчитайте качество и визуализируйте результаты. Можно ли сказать что алгоритм сильно зависит от известных начальных объектов? Есть ли класс, для которого это больше всего заметно?